In [21]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [22]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [23]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [24]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [25]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [26]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords as sw
stopwords = sw.words('english')

[nltk_data] Downloading package stopwords to /home/gg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
def tokenizer(text):
    return text.split()

In [28]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [29]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid = {'vect__ngram_range': [(2,2)],
              'vect__stop_words': [stopwords, None],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter,
                                  tokenizer_lemma],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [0.001, 0.01]}

pipe = Pipeline([
                 ('vect', count),
                 ('clf', LogisticRegression(random_state=42))
                ])

model = GridSearchCV(pipe, param_grid, scoring='neg_log_loss', cv=10, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  4.9min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(2, 2)], 'vect__tokenizer': [<function tokenizer at 0x7fdb59d3c8c0>, <function tokenizer_porter at 0x7fdb86cea7d0>, <function tokenizer_lemma at 0x7fdb4c657668>], 'clf__penalty': ['l1', 'l2'], 'clf__C': [0.001, 0.01], 'vect__stop_words': [[u'i', u'me', u'my', u'myse...uldn't", u'wasn', u"wasn't", u'weren', u"weren't", u'won', u"won't", u'wouldn', u"wouldn't"], None]},
       pre_dispatch='2*n_jobs', refit=True, return_t

In [31]:
print(model.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        s...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [32]:
print(model.best_params_)

{'vect__ngram_range': (2, 2), 'vect__tokenizer': <function tokenizer_porter at 0x7fdb86cea7d0>, 'clf__penalty': 'l2', 'clf__C': 0.01, 'vect__stop_words': None}


In [33]:
print(model.best_score_)

-0.955032009791158


In [34]:
y_pred_train = model.best_estimator_.predict(X_train)

In [35]:
from sklearn.metrics import confusion_matrix
conf_matrix_train = confusion_matrix(y_true=y_train, y_pred=y_pred_train)
print(conf_matrix_train)

[[5240  106  184]
 [1744 1974  226]
 [1536  123 2572]]


In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

0.7140459686245896

In [37]:
y_pred_test = model.best_estimator_.predict(X_test)

In [38]:
conf_matrix_test = confusion_matrix(y_true=y_test, y_pred=y_pred_test)
print(conf_matrix_test)

[[2135   81  154]
 [ 981  565  145]
 [ 914  116  783]]


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_test)

0.5929519918283963

In [40]:
import pickle
import os
pickle.dump(model.best_estimator_,
           open(os.path.join('./pkl_objects', 'bag_2gram_lr_pipe.pkl'), 'wb'),
           protocol=2)